#### Copyright 2018 Google LLC.

In [0]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Feature Extraction Using a Pretrained Model

One thing that is commonly done in computer vision is to take a model trained on a very large dataset, run it on your own, smaller dataset, and extract the intermediate representations (features) that the model generates. These representations are frequently informative for your own computer vision task, even though the task may be quite different from the problem that the original model was trained on. This versatility and repurposability of convnets is one of the most interesting aspects of deep learning.

In our case, we will use the [Inception V3 model](https://arxiv.org/abs/1512.00567) developed at Google, and pre-trained on [ImageNet](http://image-net.org/), a large dataset of web images (1.4M images and 1000 classes). This is a powerful model; let's see what the features that it has learned can do for our cat vs. dog problem.

First, we need to pick which intermediate layer of Inception V3 we will use for feature extraction. A common practice is to use the output of the very last layer before the `Flatten` operation, the so-called "bottleneck layer." The reasoning here is that the following fully connected layers will be too specialized for the task the network was trained on, and thus the features learned by these layers won't be very useful for a new task. The bottleneck features, however, retain much generality.

Let's instantiate an Inception V3 model preloaded with weights trained on ImageNet:


In [1]:
import os

from tensorflow.keras import layers
from tensorflow.keras import Model

Now let's download the weights:

In [2]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2019-11-18 04:22:49--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 2404:6800:4008:c01::80
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   137MB/s    in 0.6s    

2019-11-18 04:22:50 (137 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



By specifying the include_top=False argument, we load a network that doesn't include the classification layers at the top—ideal for feature extraction. input_shpe can be used only when include-top=false.


In [3]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(
    input_shape=(300, 150, 3), include_top=False, weights=None)
pre_trained_model.load_weights(local_weights_file)

W1118 04:22:55.926901 140112643757952 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/resource_variable_ops.py:1630: calling __init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Let's make the model non-trainable, since we will only use it for feature extraction; we won't update the weights of the pretrained model during training.

"freeze" a layer means to exclude it from training, i.e. its weights will never be updated. 

You can pass a trainable argument (boolean) to a layer constructor to set a layer to be non-trainable:

In [0]:
for layer in pre_trained_model.layers:
  layer.trainable = False

The layer we will use for feature extraction in Inception v3 is called `mixed7`. It is not the bottleneck of the network, but we are using it to keep a sufficiently large feature map (7x7 in this case). (Using the bottleneck layer would have resulting in a 3x3 feature map, which is a bit small.) Let's get the output from `mixed7`:

In [5]:
last_layer = pre_trained_model.get_layer('mixed7')
print 'last layer output shape:', last_layer.output_shape
last_output = last_layer.output

last layer output shape: (None, 17, 7, 768)


Now let's stick a fully connected classifier on top of `last_output`:

In [6]:
from tensorflow.keras.optimizers import RMSprop

# Flatten the output layer to 1 dimension
x = layers.Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = layers.Dense(1024, activation='relu')(x)
# Add a dropout rate of 0.2
x = layers.Dropout(0.2)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(1, activation='sigmoid')(x)

# Configure and compile the model
model = Model(pre_trained_model.input, x)
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.0001),
              metrics=['acc'])

W1118 04:23:21.179666 140112643757952 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [8]:
import os
import zipfile

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# local_zip = '/tmp/cats_and_dogs_filtered.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/tmp')
# zip_ref.close()



# Define our example directories and files
base_dir = '/content/drive/My Drive/Colab Notebooks/currency_images/'
train_dir = os.path.join(base_dir, 'train2000')
validation_dir = os.path.join(base_dir, 'test2000')

# Directory with our training fake pictures
train_fake_dir = os.path.join(train_dir, 'fake_2000')

# Directory with our training orignal pictures
train_orig_dir = os.path.join(train_dir, 'original_2000')

# Directory with our validation fake pictures
validation_fake_dir = os.path.join(train_dir, 'fake_2000','TrainFake')

# Directory with our validation original pictures
validation_orig_dir = os.path.join(train_dir, 'original_2000','output')

train_fake_fnames = os.listdir(train_fake_dir)
train_orig_fnames = os.listdir(train_orig_dir)

# Add our data-augmentation parameters to ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir, # This is the source directory for training images
        target_size=(300, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(300, 150),
        batch_size=20,
        class_mode='binary')


Found 12587 images belonging to 2 classes.
Found 10 images belonging to 2 classes.


Finally, let's train the model using the features we extracted. We'll train on all 2000 images available, for 3 epochs, and validate on all  test images.

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=1)


NameError: ignored

In [0]:
model.save("/content/drive/My Drive/Colab Notebooks/model_fake_original_detectionimages_50epoch.h5")

In [10]:

from tensorflow.keras.models import load_model
#from keras.models import load_model
import cv2
import numpy as np

model_new = load_model('/content/drive/My Drive/Colab Notebooks/model_fake_original_10000images_50epoch.h5')

#model_new.compile(loss='binary_crossentropy',
#               optimizer='rmsprop',
#               metrics=['accuracy'])

img = cv2.imread('/content/drive/My Drive/Colab Notebooks/currency_images/train2000/original_2000/output/real20.jpg')
#img = cv2.imread('/content/drive/My Drive/Colab Notebooks/currency_images/train2000/fake_train/fake5.jpg')


img = cv2.resize(img,(300,150))
img = np.reshape(img,[1,300,150,3])

classes = model_new.predict(img)


W1118 04:24:54.034998 140112643757952 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1118 04:24:54.036967 140112643757952 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W1118 04:24:54.046946 140112643757952 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/init_ops.py:97: calling __init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and wil

In [0]:
for file_name in os.listdir("/content/drive/My Drive/Colab Notebooks/currency_images/train2000/fake_2000/output/"):
  img = cv2.imread('/content/drive/My Drive/Colab Notebooks/currency_images/train2000/fake_2000/output/' + file_name)
  img = cv2.resize(img,(300,150))
  img = np.reshape(img,[1,300,150,3])
  classes = model_new.predict(img)
  for item in classes[:10]:
    if (item[0]>.799):
      print("Fake")
    else:
      print("Real")

Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Fake
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real
Real


function

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
from tensorflow.keras.models import load_model
#from keras.models import load_model
import cv2
import numpy as np 
model_new = load_model('/content/drive/My Drive/Colab Notebooks/model_fake_original_10000images_50epoch.h5')



In [0]:
#image_path ='/content/drive/My Drive/Colab Notebooks/currency_images/train2000/original_2000/output/real3.jpg'
image_path ='/content/drive/My Drive/Colab Notebooks/currency_images/train2000/fake_2000/output/fake25.jpg'
#img = cv2.imread('/content/drive/My Drive/Colab Notebooks/currency_images/train2000/fake_train/fake5.jpg')
#img = cv2.resize(img,(300,150))
#img = np.reshape(img,[1,300,150,3])
#classes = model_new.predict(img)

def funtest(image_path):
  #img = cv2.imread('/content/drive/My Drive/Colab Notebooks/currency_images/train2000/original_2000/output/fake50.jpg')
  img = cv2.imread(image_path)
  img = cv2.resize(img,(300,150))
  img = np.reshape(img,[1,300,150,3])
  classes = model_new.predict(img)
  for item in classes[:10]:
    if (item[0]>.799):
      print("Fake")
    else:
      print("Real")
funtest(image_path)

Real


You can see that we reach a validation accuracy of 88–90% 


very quickly. This is much better than the small model we trained from scratch.

In [0]:
pip install flask-ngrok

  Created wheel for flask-ngrok: filename=flask_ngrok-0.0.25-cp27-none-any.whl size=3043 sha256=074010b9f90fe5b9a3aef3f2c6b941741b422f0bd3c7c3e85540764ebd14f396
  Stored in directory: /root/.cache/pip/wheels/4b/52/9f/3f85c132d06485491eb5572de5eb6be2c7e3559409073ee78c
Successfully built flask-ngrok


In [0]:
!pip install flask==0.12.2

     |████████████████████████████████| 92kB 5.2MB/s 
  Found existing installation: Flask 1.1.1
    Uninstalling Flask-1.1.1:
      Successfully uninstalled Flask-1.1.1


In [0]:
from flask import Flask
#from flask_ngrok import run_with_ngrok
app = Flask(__name__)
#run_with_ngrok(app)  # Start ngrok when app is run
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

@app.route("/")
def hello():
   return "Hello World!"

if __name__ == '__main__':
   app.run(host='0.0.0.0')

I1103 08:20:57.987487 139669717890944 _internal.py:122]  * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)


172.28.0.2


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Further Improving Accuracy with Fine-Tuning

In our feature-extraction experiment, we only tried adding two classification layers on top of an Inception V3 layer. The weights of the pretrained network were not updated during training. One way to increase performance even further is to "fine-tune" the weights of the top layers of the pretrained model alongside the training of the top-level classifier. A couple of important notes on fine-tuning:

- **Fine-tuning should only be attempted *after* you have trained the top-level classifier with the pretrained model set to non-trainable**. If you add a randomly initialized classifier on top of a pretrained model and attempt to train all layers jointly, the magnitude of the gradient updates will be too large (due to the random weights from the classifier), and your pretrained model will just forget everything it has learned.
- Additionally, we **fine-tune only the *top layers* of the pre-trained model** rather than all layers of the pretrained model because, in a convnet, the higher up a layer is, the more specialized it is. The first few layers in a convnet learn very simple and generic features, which generalize to almost all types of images. But as you go higher up, the features are increasingly specific to the dataset that the model is trained on. The goal of fine-tuning is to adapt these specialized features to work with the new dataset.

All we need to do to implement fine-tuning is to set the top layers of Inception V3 to be trainable, recompile the model (necessary for these changes to take effect), and resume training. Let's unfreeze all layers belonging to the `mixed7` module—i.e., all layers found after `mixed6`—and recompile the model:

In [0]:
from tensorflow.keras.optimizers import SGD

unfreeze = False

# Unfreeze all models after "mixed6"
for layer in pre_trained_model.layers:
  if unfreeze:
    layer.trainable = True
  if layer.name == 'mixed6':
    unfreeze = True

# As an optimizer, here we will use SGD 
# with a very low learning rate (0.00001)
model.compile(loss='binary_crossentropy',
              optimizer=SGD(
                  lr=0.00001, 
                  momentum=0.9),
              metrics=['acc'])

Now let's retrain the model. We'll train on all 2000 images available, for 50 epochs, and validate on all 1,000 test images. (This may take 15-20 minutes to run.)

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=50,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)

We are seeing a nice improvement, with the validation loss going from ~1.7 down to ~1.2, and accuracy going from 88% to 92%. That's a 4.5% relative improvement in accuracy.

Let's plot the training and validation loss and accuracy to show it conclusively:

In [0]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

Congratulations! Using feature extraction and fine-tuning, you've built an image classification model that can identify cats vs. dogs in images with over 90% accuracy.

## Clean Up

Run the following cell to terminate the kernel and free memory resources:

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

In [0]:
import socket
print(socket.gethostbyname(socket.getfqdn(socket.gethostname())))

from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World!"

import threading
threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':80}).start() 

172.28.0.2
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:80/ (Press CTRL+C to quit)


In [0]:
!pip install flask-ngrok


  Created wheel for flask-ngrok: filename=flask_ngrok-0.0.25-cp27-none-any.whl size=3043 sha256=571fcbf3a066c785639dc7e984c0bbc0ba4c57fc2b18d933202c0f61ee571ca1
  Stored in directory: /root/.cache/pip/wheels/4b/52/9f/3f85c132d06485491eb5572de5eb6be2c7e3559409073ee78c
Successfully built flask-ngrok


SyntaxError: ignored